## Ingeniería de Variables


Este notebook proporciona una guía completa y paso a paso sobre la ingeniería de variables, una etapa crucial en el preprocesamiento de datos para modelos de machine learning. El proceso de ingeniería de variables implica transformar los datos brutos en un formato más adecuado y eficaz para el modelado, mejorando así la precisión y eficiencia de los modelos de aprendizaje automático.

Pasos a seguir:

**Manejo de Valores Nulos**: 

Reemplazamos los valores nulos en las variables numéricas con la mediana para mantener la integridad de la distribución de los datos.

**Creación de Nuevas Variables**: 

A través de diversas técnicas, generamos nuevas variables a partir de las existentes, enriqueciendo así el conjunto de datos con información potencialmente valiosa para los modelos predictivos.

**Preprocesamiento de Datos**:

Escalado de Variables Numéricas: Normalizamos o estandarizamos las variables numéricas para que contribuyan de manera equitativa al modelo.

Imputación y Codificación de Variables Categóricas: Utilizamos técnicas como el Simple Imputer para manejar valores faltantes y aplicamos One-Hot Encoding para convertir variables categóricas en un formato que sea más adecuado para el modelado.

Pimero importamos las librerias que vamos a necesitar.

In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
import pickle
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

Creamos distintas funciones que explicaremos y utilizaremos más adelante.

In [2]:
def calculate_risk_score(row):
    # Mayor puntuación para ingresos más bajos
    income_score = 1 - row['income']

    # Ajustar el límite de crédito para que valores más altos resulten en puntuaciones más altas
    # Puedes elegir el mejor método de ajuste según tu análisis
    credit_score = row['proposed_credit_limit'] / 2000  # Escala el crédito al rango [0.1, 1]

    return income_score * credit_score

def categorize_income_age(row):
    if row['customer_age'] < 30:
        if row['income'] > 0.7:
            return 'young_high_income'
        elif row['income'] < 0.3:
            return 'young_low_income'
        else:
            return 'young_middle_income'
    elif row['customer_age'] >= 60:
        if row['income'] > 0.7:
            return 'senior_high_income'
        elif row['income'] < 0.3:
            return 'senior_low_income'
        else:
            return 'senior_middle_income'
    else:
        if row['income'] > 0.7:
            return 'adult_high_income'
        elif row['income'] < 0.3:
            return 'adult_low_income'
        else:
            return 'adult_middle_income'
        
def categorize_current_address_stability(row):
    if row['current_address_months_count'] >= 36:  
        return 'high_stability'
    elif 12 <= row['current_address_months_count'] < 36:  
        return 'medium_stability'
    elif 0 <= row['current_address_months_count'] < 12:  
        return 'low_stability'
    else: 
        return 'unknown'

def calculate_email_score(row):
    score = 0
    # Verificar si el email es de un dominio pagado (email_is_free es 1)
    if row['email_is_free'] == 1:
        score += 1  # Un punto por ser dominio pagado

        # Verificar la similitud entre el nombre y el email
        if row['name_email_similarity'] > 0.5:  # Alta similitud
            score += 1  # Un punto adicional por alta similitud
        elif row['name_email_similarity'] > 0.3:  # Similitud moderada (pero no alta)
            score += 0.5  # Medio punto por similitud moderada
    else:
        # Si el dominio es gratuito, solo se considera la similitud
        if row['name_email_similarity'] > 0.5:
            score += 1  # Un punto por alta similitud

    return score

def replace_with_median(df, variables):
    for variable in variables:
        median_value = df[variable].median()

        if variable == 'intended_balcon_amount':
            df.loc[df[variable] < 0, variable] = median_value
        else:
            df.loc[df[variable] == -1, variable] = median_value
    return df

def apply_risk_score(df):
    df['income_Risk_Score'] = df.apply(calculate_risk_score, axis=1)
    return df

def apply_income_age_category(df):
    df['income_age_category'] = df.apply(categorize_income_age, axis=1)
    return df
    
def apply_current_address_stability(df):
    df['current_address_stability'] = df.apply(categorize_current_address_stability, axis=1)
    return df
    
def apply_email_score(df):
    df['email_score'] = df.apply(calculate_email_score, axis=1)
    return df

def preparar_datos(df, eliminar_columnas=False):
    columnas_a_eliminar = ['income', 'customer_age', 'proposed_credit_limit', 'current_address_months_count', 'email_is_free', 'name_email_similarity']
    if eliminar_columnas:
        df = df.drop(columns=columnas_a_eliminar)
    X = df.drop('fraud_bool', axis=1)
    y = df['fraud_bool']
    return X, y
    
def transformar_y_dataframe(X, preprocessor):
    X_transformed = preprocessor.transform(X)
    column_names = preprocessor.get_feature_names_out()
    X_df = pd.DataFrame(X_transformed, columns=column_names)

    return X_df

Leemos los pickles que guardamos en el notebook anterior. De esta forma conservan el formato aplicado.

In [3]:
pd_fraud_train = pd.read_pickle("../data/pd_fraud_train.pkl")

In [4]:
pd_fraud_test = pd.read_pickle("../data/pd_fraud_test.pkl")


Hacemos una lista que contiene las variables que tienen valores missings.

In [5]:
variables_con_missings=['prev_address_months_count','current_address_months_count','intended_balcon_amount', 'bank_months_count', 'session_length_in_minutes', 'device_distinct_emails_8w']

Aplicamos la función replace_with_median a la cual fue diseñada y trabaja de la siguiente manera:

Se pasa un DataFrame (df) y una lista de nombres de variables en las cuales se quiere realizar la corrección.

Para cada variable en esta lista, la función calcula primero la mediana de esa variable en el DataFrame.

Luego, realiza dos tipos de correcciones dependiendo del nombre de la variable:

Si la variable es 'intended_balcon_amount', la función reemplaza todos los valores que son menores a 0 con la mediana calculada. Esto es debido a que en esta columna todos los valores negativos son nulos.

Para las demás variables en la lista, la función reemplaza los valores que son exactamente -1 con la mediana. Es estas columnas todos los valores iguales a -1 son nulos.

In [6]:

pd_fraud_train = replace_with_median(pd_fraud_train, variables_con_missings)

In [7]:
pd_fraud_test = replace_with_median(pd_fraud_test, variables_con_missings)

Aqui vamos a crear la primera variable. Utilizaremos la función calculate_risk_score. 

La función calculate_risk_score se enfoca en calcular una puntuación de riesgo utilizando dos parámetros financieros clave: los ingresos y el límite de crédito propuesto. La finalidad es obtener un indicador de riesgo que integre y pondera adecuadamente estos dos aspectos, los cuales contribuyen de manera distinta a la puntuación global.

-Ingresos (Income): Aquí, los ingresos están expresados en deciles, con valores que varían entre 0.1 y 0.9. La función asigna puntuaciones inversamente proporcionales a los ingresos; es decir, a ingresos más bajos (más cercanos a 0.1) corresponden puntuaciones más altas. Esto se logra mediante la sustracción del valor de los ingresos de 1. Por lo tanto, ingresos menores se traducen en puntuaciones más elevadas, sugiriendo un riesgo más alto.


-Límite de Crédito Propuesto: Para esta variable, se realiza un ajuste de manera que valores más altos en el límite de crédito se reflejen en puntuaciones más altas. La función utiliza una fórmula de normalización, dividiendo el límite de crédito propuesto por 2000. Este ajuste es crucial para estandarizar el rango de los valores de límite de crédito, que se encuentran entre 200 y 2000, obteniendo así un rango de puntuación manejable y consistente.

Al estar ambas variables entre 0.1 y 1 ninguna tiene más peso que otra.

Finalmente, la función devuelve el producto de estas dos puntuaciones, integrando los factores de ingresos y límite de crédito en una única métrica.De esta forma obtenemos una valoración comprensiva del riesgo asociado a un individuo o transacción, usando varias variables financieras. Por último elaboramos una función para aplicar esta función al dataset de train y al test.


In [8]:
pd_fraud_train = apply_risk_score(pd_fraud_train)


In [9]:
pd_fraud_test = apply_risk_score(pd_fraud_test)

In [10]:
pd_fraud_train

,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,velocity_6h,...,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,month,fraud_bool,income_Risk_Score
40088,0.8,0.414604,-1,11,30,0.011261,-0.830703,AB,785,806.667362,...,1000.0,0,INTERNET,7.686806,windows,1,1,7,0,0.100
823737,0.9,0.778614,-1,65,30,0.007631,-0.830703,AB,987,4101.304695,...,500.0,0,INTERNET,4.379858,other,1,1,5,0,0.025
915672,0.1,0.748364,-1,143,50,0.013925,-0.830703,AB,4153,6082.720553,...,200.0,0,INTERNET,2.946081,other,1,1,4,0,0.090
582312,0.7,0.542464,-1,199,40,0.019345,-0.830703,AC,1063,5332.841790,...,200.0,0,INTERNET,2.373413,other,0,1,6,0,0.030
603880,0.7,0.289721,-1,38,40,0.000391,-0.830703,AC,1385,3118.605192,...,200.0,0,INTERNET,1.032444,windows,0,1,6,0,0.030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216662,0.7,0.773012,28,13,30,0.005354,50.883241,AA,2433,7070.604726,...,500.0,0,INTERNET,5.015641,other,0,1,3,0,0.075
72768,0.8,0.385426,236,8,20,0.006199,-0.830703,AC,381,1466.619966,...,200.0,0,INTERNET,0.631367,linux,1,1,7,0,0.020
105656,0.1,0.381497,150,4,50,0.013224,-0.830703,AB,1618,8040.139913,...,200.0,0,INTERNET,10.599065,linux,1,1,3,0,0.090
624415,0.6,0.416131,-1,45,20,0.011576,49.466975,AA,1349,11645.859403,...,500.0,0,INTERNET,4.237907,windows,1,1,0,0,0.100


La segunda variable que creamos se hace a través de la función categorize_income_age. 

La función categorize_income_age clasifica a los clientes en distintas categorías basadas en su edad e ingresos. Esta clasificación puede ser útil por varios aspectos:

Segmentación de Clientes: Crea segmentos específicos de clientes como 'joven con ingresos altos', 'adulto con ingresos medios', 'senior con ingresos bajos', etc. Esto es especialmente valioso para modelos que necesitan comprender y predecir comportamientos o preferencias que pueden ser distintos en cada grupo.

Reducción de la Complejidad del Modelo: Al convertir variables numéricas continuas (como edad e ingresos) en categorías, se simplifica el espacio de características. Esto puede hacer que los patrones en los datos sean más fáciles de detectar para ciertos modelos, especialmente aquellos que no manejan bien las interacciones complejas o las relaciones no lineales.

Esta segmentación permite identificar tendencias y patrones que son específicos de ciertos grupos de edad e ingresos, lo cual puede ser crucial para el análisis de riesgo crediticio.

Utilizamos la funcion apply_income_age_category para aplicar esta función en train y en test.


In [11]:

pd_fraud_train = apply_income_age_category(pd_fraud_train)


In [12]:
pd_fraud_test = apply_income_age_category(pd_fraud_test)

In [13]:
pd_fraud_train

,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,velocity_6h,...,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,month,fraud_bool,income_Risk_Score,income_age_category
40088,0.8,0.414604,-1,11,30,0.011261,-0.830703,AB,785,806.667362,...,0,INTERNET,7.686806,windows,1,1,7,0,0.100,adult_high_income
823737,0.9,0.778614,-1,65,30,0.007631,-0.830703,AB,987,4101.304695,...,0,INTERNET,4.379858,other,1,1,5,0,0.025,adult_high_income
915672,0.1,0.748364,-1,143,50,0.013925,-0.830703,AB,4153,6082.720553,...,0,INTERNET,2.946081,other,1,1,4,0,0.090,adult_low_income
582312,0.7,0.542464,-1,199,40,0.019345,-0.830703,AC,1063,5332.841790,...,0,INTERNET,2.373413,other,0,1,6,0,0.030,adult_high_income
603880,0.7,0.289721,-1,38,40,0.000391,-0.830703,AC,1385,3118.605192,...,0,INTERNET,1.032444,windows,0,1,6,0,0.030,adult_high_income
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216662,0.7,0.773012,28,13,30,0.005354,50.883241,AA,2433,7070.604726,...,0,INTERNET,5.015641,other,0,1,3,0,0.075,adult_high_income
72768,0.8,0.385426,236,8,20,0.006199,-0.830703,AC,381,1466.619966,...,0,INTERNET,0.631367,linux,1,1,7,0,0.020,young_high_income
105656,0.1,0.381497,150,4,50,0.013224,-0.830703,AB,1618,8040.139913,...,0,INTERNET,10.599065,linux,1,1,3,0,0.090,adult_low_income
624415,0.6,0.416131,-1,45,20,0.011576,49.466975,AA,1349,11645.859403,...,0,INTERNET,4.237907,windows,1,1,0,0,0.100,young_middle_income


La tercera variable la creamos con la función categorize_current_address_stability. 


La función categorize_current_address_stability categoriza a los clientes según la estabilidad de su dirección actual, basándose en la cantidad de meses que han vivido en ella. Esta clasificación es útil para proporcionar una medida de estabilidad residencial, la cual puede ser un indicador importante en varios contextos, especialmente en el análisis de crédito y riesgo financiero. 

Alta Estabilidad (high_stability): Se asigna esta categoría a aquellos clientes que han vivido en su dirección actual por más de 3 años (36 meses). Esto sugiere un alto nivel de estabilidad residencial.

Estabilidad Media (medium_stability): Esta categoría se aplica a los clientes cuya duración en la dirección actual es de entre 1 y 3 años (12 a 35 meses). Indica un nivel moderado de estabilidad.

Baja Estabilidad (low_stability): Se categoriza como baja estabilidad a los clientes que han vivido en su dirección actual menos de 1 año (0 a 11 meses). Esto podría ser un signo de inestabilidad residencial.

Desconocido (unknown): Esta categoría se utiliza para casos en los que el tiempo en la dirección actual no está disponible o presenta valores inusuales. Esto podría incluir valores faltantes o datos erróneos.



In [14]:
pd_fraud_train = apply_current_address_stability(pd_fraud_train)


In [15]:
pd_fraud_test = apply_current_address_stability(pd_fraud_test)

In [16]:
pd_fraud_train

,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,velocity_6h,...,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,month,fraud_bool,income_Risk_Score,income_age_category,current_address_stability
40088,0.8,0.414604,-1,11,30,0.011261,-0.830703,AB,785,806.667362,...,INTERNET,7.686806,windows,1,1,7,0,0.100,adult_high_income,low_stability
823737,0.9,0.778614,-1,65,30,0.007631,-0.830703,AB,987,4101.304695,...,INTERNET,4.379858,other,1,1,5,0,0.025,adult_high_income,high_stability
915672,0.1,0.748364,-1,143,50,0.013925,-0.830703,AB,4153,6082.720553,...,INTERNET,2.946081,other,1,1,4,0,0.090,adult_low_income,high_stability
582312,0.7,0.542464,-1,199,40,0.019345,-0.830703,AC,1063,5332.841790,...,INTERNET,2.373413,other,0,1,6,0,0.030,adult_high_income,high_stability
603880,0.7,0.289721,-1,38,40,0.000391,-0.830703,AC,1385,3118.605192,...,INTERNET,1.032444,windows,0,1,6,0,0.030,adult_high_income,high_stability
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216662,0.7,0.773012,28,13,30,0.005354,50.883241,AA,2433,7070.604726,...,INTERNET,5.015641,other,0,1,3,0,0.075,adult_high_income,medium_stability
72768,0.8,0.385426,236,8,20,0.006199,-0.830703,AC,381,1466.619966,...,INTERNET,0.631367,linux,1,1,7,0,0.020,young_high_income,low_stability
105656,0.1,0.381497,150,4,50,0.013224,-0.830703,AB,1618,8040.139913,...,INTERNET,10.599065,linux,1,1,3,0,0.090,adult_low_income,low_stability
624415,0.6,0.416131,-1,45,20,0.011576,49.466975,AA,1349,11645.859403,...,INTERNET,4.237907,windows,1,1,0,0,0.100,young_middle_income,high_stability


La última columa se crea a partir de la función calculate_email_score.

La función calculate_email_score asigna una puntuación basada en dos factores relacionados con la dirección de correo electrónico de un cliente: si el dominio del correo es de pago y el grado de similitud entre el nombre del cliente y su dirección de correo electrónico. La puntuación se calcula de la siguiente manera:

Dominio de Pago: Si el correo electrónico del cliente proviene de un dominio pagado (indicado por email_is_free igual a 1), se añade un punto a la puntuación. Los correos electrónicos de dominios pagados a menudo se consideran más profesionales o serios que los de dominios gratuitos.

Similitud entre Nombre y Correo Electrónico:

Alta Similitud: Si la similitud entre el nombre del cliente y su dirección de correo electrónico (representada por name_email_similarity) es mayor a 0.5, se añade un punto adicional. Esto indica que el correo electrónico es personal y específico para el usuario.

Similitud Moderada: Si la similitud está por encima de 0.3 pero no supera el 0.5, se añade medio punto. Esto sugiere una cierta relación entre el nombre del usuario y su correo electrónico, aunque no tan fuerte como en el caso anterior.

Dominio Gratuito: Si el correo electrónico es de un dominio gratuito (es decir, email_is_free no es 1), la función solo considera la similitud entre el nombre y el correo electrónico para la puntuación. En este caso, una alta similitud (más de 0.5) añade un punto a la puntuación.

Esta nueva variable aporta una dimensión adicional al perfil del cliente, proporcionando una métrica que puede ser relevante para la autenticidad del cliente, la prevención del fraude y la segmentación del mercado.

En este contexto donde el fraude es una preocupación, una puntuación baja podría ser un indicador de riesgo.

Un correo electrónico que parece más personalizado o profesional (dominio pagado, alta similitud con el nombre del usuario) podría considerarse un indicador de autenticidad o seriedad del cliente.




In [17]:
pd_fraud_train = apply_email_score(pd_fraud_train)

In [18]:
pd_fraud_test = apply_email_score(pd_fraud_test)

In [19]:
pd_fraud_train

,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,velocity_6h,...,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,month,fraud_bool,income_Risk_Score,income_age_category,current_address_stability,email_score
40088,0.8,0.414604,-1,11,30,0.011261,-0.830703,AB,785,806.667362,...,7.686806,windows,1,1,7,0,0.100,adult_high_income,low_stability,1.5
823737,0.9,0.778614,-1,65,30,0.007631,-0.830703,AB,987,4101.304695,...,4.379858,other,1,1,5,0,0.025,adult_high_income,high_stability,1.0
915672,0.1,0.748364,-1,143,50,0.013925,-0.830703,AB,4153,6082.720553,...,2.946081,other,1,1,4,0,0.090,adult_low_income,high_stability,1.0
582312,0.7,0.542464,-1,199,40,0.019345,-0.830703,AC,1063,5332.841790,...,2.373413,other,0,1,6,0,0.030,adult_high_income,high_stability,2.0
603880,0.7,0.289721,-1,38,40,0.000391,-0.830703,AC,1385,3118.605192,...,1.032444,windows,0,1,6,0,0.030,adult_high_income,high_stability,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216662,0.7,0.773012,28,13,30,0.005354,50.883241,AA,2433,7070.604726,...,5.015641,other,0,1,3,0,0.075,adult_high_income,medium_stability,2.0
72768,0.8,0.385426,236,8,20,0.006199,-0.830703,AC,381,1466.619966,...,0.631367,linux,1,1,7,0,0.020,young_high_income,low_stability,1.5
105656,0.1,0.381497,150,4,50,0.013224,-0.830703,AB,1618,8040.139913,...,10.599065,linux,1,1,3,0,0.090,adult_low_income,low_stability,1.5
624415,0.6,0.416131,-1,45,20,0.011576,49.466975,AA,1349,11645.859403,...,4.237907,windows,1,1,0,0,0.100,young_middle_income,high_stability,1.5


En primer lugar vamos a preparar dos tipos de datos. Unos para modelos de árboles y emsembled y otros para el resto de modelos.
Esto se debe a que los árboles y los modelos de emsembled cuando creamos nuevas variables a partie de variables originales no es estrictamente necesario eliminar las variables originlaes debido a la capacidad del modelo para manejar relaciones no lineales y seleccionar características importantes. 

Sin embargo en los modelos de regresión si es conveniente eliminar las variables originales y por lo tanto evitar la multicolinealidad. Si el coeficiente de correlación es muy alto, al resolver la ecuación matemática de la regresión ocurre que el determinante de la matriz resultante es muy cercano a cero y su inversa es infinito, lo que supone tener unos coeficientes y un modelo inestable y dificulta la interpretación del modelo. 
En naive Bayes y en el SVC tambien es conveniente eliminarlas en principio.

Por ello preprocesaremos tanto datos para árboles de decisión y emsembled como para el resto de modelos.

Elaboramos una lista con las variables que deben estar en los modelos de regresión, NaiveBayes y SVC.

In [20]:
lista_variables_categoricas_nb_lr_svc = ["payment_type","employment_status","housing_status","phone_home_valid",
                                "phone_mobile_valid","has_other_cards","foreign_request","source","device_os","keep_alive_session", "income_age_category","current_address_stability"]

lista_variables_numericas_nb_lr_svc = [ "prev_address_months_count", "days_since_request",
                            "intended_balcon_amount", "zip_count_4w", "velocity_6h", "velocity_24h", "velocity_4w", "bank_branch_count_8w", "date_of_birth_distinct_emails_4w",
                            "credit_risk_score", "bank_months_count", "session_length_in_minutes", "device_distinct_emails_8w",
                            "month", "income_Risk_Score","email_score"]
target = 'fraud_bool'

Hacemos lo mismo para modelos de árboles de decisión y emsembled.

In [21]:
lista_variables_categoricas = ["payment_type","employment_status","housing_status","phone_home_valid",
                                "phone_mobile_valid","has_other_cards","email_is_free","foreign_request","source","device_os","keep_alive_session", "income_age_category","current_address_stability"]

lista_variables_numericas = ["income", "customer_age", "proposed_credit_limit", "current_address_months_count","name_email_similarity", "prev_address_months_count", "days_since_request",
                            "intended_balcon_amount", "zip_count_4w", "velocity_6h", "velocity_24h", "velocity_4w", "bank_branch_count_8w", "date_of_birth_distinct_emails_4w",
                            "credit_risk_score", "bank_months_count", "session_length_in_minutes", "device_distinct_emails_8w",
                            "month", "income_Risk_Score","email_score"]
target = 'fraud_bool'

Hacemos un preprocesador para ambas situaciones. Ambos preprocesadores hacen exactamente lo mismo, preparan las variables categóricas y numéricas para ser utilizadas en un modelo predictivo.

**Transformador para Variables Numéricas** (num_transformer):

Utiliza un StandardScaler para estandarizar las variables numéricas. La estandarización reescala los datos para que tengan una media de 0 y una desviación estándar de 1, lo que es útil para muchos algoritmos de machine learning.


**Transformador para Variables Categóricas** (cat_transformer):

Aplica un SimpleImputer que reemplaza los valores faltantes en variables categóricas con la cadena 'Unknown'. Esto es útil para manejar los valores perdidos.
Utiliza OneHotEncoder para convertir variables categóricas en una forma que los algoritmos de machine learning pueden utilizar. El OneHotEncoder crea una nueva columna binaria para cada categoría de cada variable categórica.

Este código prepara el conjunto de datos para el modelado, asegurando que las variables numéricas y categóricas sean tratadas de manera adecuada y estén listas para ser utilizadas por un algoritmo de machine learning.

In [22]:
num_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

In [23]:
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [24]:
preprocessor_nb_lr_svc = ColumnTransformer(
    transformers=[
        ('cat', cat_transformer,lista_variables_categoricas_nb_lr_svc),
        ('num', num_transformer,lista_variables_numericas_nb_lr_svc )
    ]
)

In [25]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', cat_transformer,lista_variables_categoricas),
        ('num', num_transformer,lista_variables_numericas )
    ]
)

Aquí vamos a preparar los datos de los modelos de regresión,NaiveBayes y SVC.

Utilizamos la función preparar_datos que al poner "eliminar_columnas=True" nos elimina las variables originales utilizadas para crear nuevas variables y nos separan tanto en el train como en el test las variables independientes y la variable objetivo.

In [26]:
X_train_nb_lr_svc, y_train_nb_lr_svc = preparar_datos(pd_fraud_train, eliminar_columnas=True)
X_test_nb_lr_svc, y_test_nb_lr_svc = preparar_datos(pd_fraud_test, eliminar_columnas=True)

Enrenamos el preprocesador.

In [27]:
preprocessor_nb_lr_svc.fit(X_train_nb_lr_svc)

ColumnTransformer(transformers=[('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='Unknown',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['payment_type', 'employment_status',
                                  'housing_status', 'phone_home_valid',
                                  'phone_mobile_valid', 'has_other_cards',
                                  'foreign_request', 'source', 'device_os',
                                  'keep_alive_session', 'inc...
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 ['prev_address_months_count',
                                  'days_since_request',
                                  'intended_balcon_amount', 'zip_count_4w',
                                  'velocity_6h', 'velocity_24h', 'velocity_4w',
                                  'bank_branch_count_8w',
                                  'date_of_birth_distinct_emails_4w',
                                  'credit_risk_score', 'bank_months_count',
                                  'session_length_in_minutes',
                                  'device_distinct_emails_8w', 'month',
                                  'income_Risk_Score', 'email_score'])])

Guardamos este preprocesador entrenado.

In [28]:
with open('../models/preprocessor_nb_lr_svc.pickle', 'wb') as f:
    pickle.dump(preprocessor_nb_lr_svc, f)

Por último utilizamos la función transformar_y_dataframe para transformar las variables según los criterios especificados en el preprocesador y convertimos el resultado de la transformación en un Dataframe utilizando el nombre de las características generadas por el preprocesador.

In [29]:
X_train_nb_lr_svc_df = transformar_y_dataframe(X_train_nb_lr_svc, preprocessor_nb_lr_svc)
X_test_nb_lr_svc_df = transformar_y_dataframe(X_test_nb_lr_svc, preprocessor_nb_lr_svc)

Hacemos lo mismo para los modelos de árboles de decisión y emsembled pero en este caso manteniendo las variables originales.

In [30]:
X_train, y_train = preparar_datos(pd_fraud_train, eliminar_columnas=False)
X_test, y_test = preparar_datos(pd_fraud_test, eliminar_columnas=False)

In [31]:
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='Unknown',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['payment_type', 'employment_status',
                                  'housing_status', 'phone_home_valid',
                                  'phone_mobile_valid', 'has_other_cards',
                                  'email_is_free', 'foreign_request', 'source',
                                  'device_os', 'keep_ali...
                                  'name_email_similarity',
                                  'prev_address_months_count',
                                  'days_since_request',
                                  'intended_balcon_amount', 'zip_count_4w',
                                  'velocity_6h', 'velocity_24h', 'velocity_4w',
                                  'bank_branch_count_8w',
                                  'date_of_birth_distinct_emails_4w',
                                  'credit_risk_score', 'bank_months_count',
                                  'session_length_in_minutes',
                                  'device_distinct_emails_8w', 'month',
                                  'income_Risk_Score', 'email_score'])])

Guardamos este preprocesador entrenado.

In [32]:
with open('../models/preprocessor.pickle', 'wb') as f:
    pickle.dump(preprocessor, f)

In [33]:
X_train_df = transformar_y_dataframe(X_train, preprocessor)
X_test_df = transformar_y_dataframe(X_test, preprocessor)

Por último guardamos todo en Pickles para mantener los cambios aplicados.

In [34]:
import pickle
with open('../data/X_train.pkl', 'wb') as f:
    pickle.dump(X_train_df, f)

with open('../data/y_train.pkl', 'wb') as f:
    pickle.dump(y_train, f)

with open('../data/X_test.pkl', 'wb') as f:
    pickle.dump(X_test_df, f)

with open('../data/y_test.pkl', 'wb') as f:
    pickle.dump(y_test, f)

In [35]:

with open('../data/X_train_nb_lr_svc.pkl', 'wb') as f:
    pickle.dump(X_train_nb_lr_svc_df, f)

with open('../data/y_train_nb_lr_svc.pkl', 'wb') as f:
    pickle.dump(y_train_nb_lr_svc, f)

with open('../data/X_test_nb_lr_svc.pkl', 'wb') as f:
    pickle.dump(X_test_nb_lr_svc_df, f)

with open('../data/y_test_nb_lr_svc.pkl', 'wb') as f:
    pickle.dump(y_test_nb_lr_svc, f)